In [ ]:
import numpy as np
import operator
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import sklearn.feature_extraction.text as fex


In [2]:
prodIdColName = "prodId"
subColName = "SubName"
sentColName = "sentence"
trainDF = pd.DataFrame({prodIdColName: [1, 1, 1,2], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['SENTтелефон', 'SENTсмартвон', 'SENT00жизнь', 'SENT2жизнь'] })

In [3]:
testDF = pd.DataFrame({prodIdColName: [3, 3, 3,4], subColName: ['телефон', 'смартвон', 'жизнь', 'жизнь'], sentColName:['3SENTтелефон', '3SENTсмартвон', '3SENT00жизнь', '3SENT2жизнь'] })

In [4]:
full = trainDF.append(testDF)

In [5]:
df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)


In [6]:
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
df_train.head()

,Unnamed: 0,PRODUCT,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Что особенно ценно - при профилактике принтера...,23.10.16 22:26:00,5.0,"хорошая скорость печати, двусторонняя печать, ...","Рядом ""на стол"" не поставишь, место много зани...",1.0,2.0,0.0
1,1,30024724,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,Он у нас три месяца в офисе стоит уже - большо...,17.04.16 11:17:42,5.0,NaN,NaN,NaN,0.0,2.0
2,2,30024725,2060202,1900,PRINTERS LASER,Kyocera,REVIEW,NaN,"Просто небо и земля с тем принтером, который у...",15.04.16 20:22:47,4.0,NaN,NaN,NaN,0.0,1.0
3,3,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,С такой вроде бы навороченной техникой все смо...,У нас в кабинете недавно его поставили. Первое...,18.04.16 20:01:57,5.0,NaN,NaN,NaN,0.0,0.0
4,4,30024727,2060101,1900,ALL-IN-ONE LASER,Kyocera,REVIEW,NaN,Очень многофункциональная штука. Есть все нео...,23.04.16 12:03:39,5.0,NaN,NaN,NaN,1.0,0.0


In [7]:
morph = pymorphy2.MorphAnalyzer()

def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 

In [8]:
#df_train[textColName]=df_train[textColName].apply(clean_text).apply(normalize)

In [42]:
import tomita
#from tparser import tomita
from tomita_parser import text_parse
import pandas as pd

In [43]:
catidNameForBuild = "SMARTPHONES"
category_data = df_train[df_train[catNameColName].str.contains(catidNameForBuild)==True]

In [44]:
textAndProd = category_data.ix[:,[textColName,tovarIdColName]]
textAndProdGrS = textAndProd.groupby(tovarIdColName )[tovarIdColName, textColName].agg(lambda x: "{%s}" % ', '.join(x)).reset_index()
print(len(textAndProd))
print(len(textAndProdGrS))

16844
2164


/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [45]:
print(len(df_train))
df_train = df_train[(df_train[tovarIdColName] != 30025022) & (df_train[tovarIdColName] != 30025023 )]
print(len(df_train))

270874
270874


In [46]:
#mask.to_csv("newProd.csv")
df_test = pd.read_csv('newProd.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)

In [15]:
colList = pd.unique(df_test[catNameColName])

In [16]:
cat = colList[0][1:]
category_data = df_train[df_train[catNameColName].str.contains(cat)==True]
len(category_data)

11694

In [17]:
#fullText = category_data.groupby([textColName,positColName,negatColName]).apply(lambda x: ' '.join(x), axis=1)
#category_data[textColName].str.append(" ")
fTextColName = 'fullText'
fTextColNameNorm = 'fullTextNorm'

In [18]:
category_data[fTextColName] = category_data[textColName].astype('str') + " "+category_data[positColName].astype('str')
category_data[fTextColName] = category_data[fTextColName]  +" "+category_data[negatColName].astype('str')

/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
category_data[fTextColNameNorm] = category_data[fTextColName].apply(clean_text).apply(normalize)

/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
len(category_data)

11694

In [21]:
category_data_test = df_test[df_test[catNameColName].str.contains(cat)==True]
category_data_test[fTextColName] = category_data_test[textColName].astype('str') + " "+category_data_test[positColName].astype('str')
category_data_test[fTextColName] = category_data_test[fTextColName]  +" "+category_data_test[negatColName].astype('str')
category_data_test[fTextColNameNorm]=category_data_test[fTextColName].apply(clean_text).apply(normalize)
textAndProdGrS_test = category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
textAndProdGrSNorm_test =  category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
textAndProdGrS = category_data.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
textAndProdGrSNorm =  category_data.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
len(textAndProdGrSNorm)

1322

In [22]:
#category_data[fTextColName].iloc[1] #03:42 №0757

In [27]:
len(textAndProdGrSNorm)

1322

In [80]:
textAndProdGrSNorm_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
textAndProdGrS_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])

len(textAndProdGrSNorm_All)

1324

In [37]:
tf_list = np.array(textAndProdGrSNorm_All)
print(tf_list.shape)
# for id_t in textAndProdGrSNorm_test[tovarIdColName]:
#     print(id_t)
#     idL = textAndProdGrSNorm_All[textAndProdGrSNorm_All[tovarIdColName] == id_t].index.tolist()
#     print("idL")
#     print(idL)
#     #print(idL.astype("int"))
#     print(textAndProdGrSNorm_All[idL])
print("true")
print(len(textAndProdGrSNorm_All)-1)
textAndProdGrSNorm_All.iloc[len(textAndProdGrSNorm_All)-1]
# indList = list()
id_tlist = textAndProdGrSNorm_test[tovarIdColName].tolist()
# #id_tlist
indList = [i for i in range(0,len(tf_list)) if tf_list[i][0] in id_tlist]
indList
#         if tf_list[i] in id_t:
#             indList.append(i)


(1324, 2)
true
1323


[1322, 1323]

In [83]:
textAndProdGrS_test['tomita'] = textAndProdGrS_test[:][fTextColName].apply(text_parse)
textAndProdGrS_test

,PRODUCT,fullText,tomita
0,30025022,"Очень порадовал аккумулятор, хватает более че...","[[АККУМУЛЯТОР, ОЧЕНЬ ПОРАДОВАЛ АККУМУЛЯТОР], [..."
1,30025023,"Отличный дизайн, яркий экран, камера четкая,з...","[[ДИЗАЙН, ОТЛИЧНЫЙ ДИЗАЙН], [ЭКРАН, ЯРКИЙ ЭКРА..."


In [ ]:
tfidf = fex.TfidfVectorizer()
tfIdfMatrix = tfidf.fit_transform(tf_list[:,1])
ftNames = tfidf.get_feature_names()
vorDic = tfidf.vocabulary
densM = tfIdfMatrix.todense()
densM.shape

In [62]:
valD = dict() 
for i in range(ind,ind+1): 
    col = densM[i] col = col.tolist() col = col[0]


SyntaxError: invalid syntax (<ipython-input-62-450679eb60d8>, line 1)

In [110]:
sublist

array(['ОЧЕНЬ ПОРАДОВАЛ АККУМУЛЯТОР',
       'ХВАТАТЬ БОЛЕЕ ЧЕМ НА СУТКИ ПРИ АКТИВНОМ ИСПОЛЬЗОВАНИЕ',
       'ИНТЕРЕСНАЯ ОБОЛОЧКА', 'ИНТЕРНЕТ ЛЕТАТЬ', 'ИДТИ БЕЗ ЛАГ',
       'АППАРАТ ПОЛНОСТЬЮ',
       'ВЫСОКОСКОРОСТНОЙ ИНТЕРНЕТ РАБОТАЕТ ТОЛЬКО С ОДНОЙ SIM КАРТА',
       'СЛОЖНО НАЙТИ АКСЕССУАР', 'ХОРОШИЙ ЗА СВОИ ДЕНЬГА',
       'ТОЛЬКО ПОПАЛСЯ ВИДИМО С БРАК', 'МАГАЗИН ЗАЧЕТНЫЙ',
       'САМ НИ ЗА ЧТО НЕ ПОМЕНЯЮ ЯБЛОЧНУЮ КОМПАНИЯ',
       'ОСТАЛЬНОЙ ЧЛЕН СЕМЬИ', 'ПРЕИМУЩЕСТВО IOS НЕ УМЕЮТ',
       'ЭТО ИДЕАЛЬНЫЙ ВАРИАНТ ДЛЯ ВАШИХ СОВРЕМЕННЫХ МАМ',
       'ТАКЖЕ ЭТО ПРЕКРАСНЫЙ ВАРИАНТ ЕСЛИ ОЧЕНЬ ХОЧЕТСЯ СМАРТФОН-ФЛАГМАН',
       'ЭТОТ ТЕЛЕФОН КУПИЛ БУКВАЛЬНО ВЧЕРА', 'Я ДОЛГО ЖДАЛ', 'ЭТА МОДЕЛЬ',
       'ДО ЭТОТ ПОЛЬЗОВАЛСЯ', 'ПОТРЯСАЮЩИЙ ТЕЛЕФОН ЗА ТАКУЮ ЦЕНА',
       'ГОД И НИКАКИХ ПРОБЛЕМ С ТЕЛЕФОНОМ НЕ БЫЛО',
       'БОЛЬШЕ ВСЕГО МНЕ НРАВИЛАСЬ АВТОНОМНОСТЬ',
       'НА НОВОЙ МОДЕЛИ ПРО АККУМУЛЯТОР СТАЛ ПРАКТИЧЕСКИ В ДВА РАЗ',
       'ТО ЕСТЬ ТЕЛЕФОН РАБОТАТЬ БУДЕТ ПО СУТИ В ДВА РАЗ',

In [107]:
sublist[0]
slAll=list()
for sl in sublist:
    #print(sl)
    #print(type(sl))
    slAll.append(normalize(sl))
   # print(sl + normalize(sl))

In [189]:
maxCount =15
print(tomitaCol)
print
for ind in indList[1:]:
    print("#"*50)
    print(ind)
    col = densM[ind]
    col = col.tolist()[0]
    valD = { ftNames[i]:v for i,v in enumerate(col) if v>0  }
    sorted_topWords = sorted(valD.items(), key=operator.itemgetter(1), reverse=True)
    sorted_topWords = [ s for s in sorted_topWords if len(s[0])>3]
    dict_r = { s[0]:s[1] for s in sorted_topWords}
 #   print(dict_r)
    print("tomita")
    tomitaD = textAndProdGrS_test[textAndProdGrS_test[tovarIdColName] == textAndProdGrS_All.iloc[ind][tovarIdColName]]['tomita']
 #   print(tomitaD.tolist())
    sublist = list([x[::,0] for x in tomitaD.values][0])
    #print(sublist)

#    print(sublist)
    set_ret = dict()
    print("#"*50)
    tomitaPairs = tomitaD.tolist()[0]

    for ts in tomitaPairs:
        ret =0
        tsn = normalize(ts[0])
        if tsn in dict_r.keys():
            ret+=dict_r[tsn]
        for tsPr in ts[1].split(" "):
            tsPrn = normalize(tsPr)
            #print(tsPrn)
            if tsPrn in dict_r.keys():
             #   print("yes")
                ret+=dict_r[tsPrn]
        set_ret[ts[1]]=ret
    print("finishrechec")
    sorted_sent = sorted(set_ret.items(), key=operator.itemgetter(1), reverse=True)
    print(sorted_sent[0:10])
    #tomitaD = tomitaD.tolist()[0]

    slAll=list()
    for sl in sublist:
        #print(sl)
        #print(type(sl))
        slAll.append(normalize(sl))
                      #s) for sls in sl.split)
    #print(slAll[1])
    for w in sorted_topWords[:maxCount]: 
        #print(w[0])

        if normalize(w[0]) in slAll:
     #       print(w)
            was = False 
            for tw in tomitaPairs:
              #  print(tw[0])
                if tw[0] == w[0].upper(): 
                    was=True 
                    print(tw[1]) 
            if was:
                print(w)

    

0    [[АККУМУЛЯТОР, ОЧЕНЬ ПОРАДОВАЛ АККУМУЛЯТОР], [...
1    [[ДИЗАЙН, ОТЛИЧНЫЙ ДИЗАЙН], [ЭКРАН, ЯРКИЙ ЭКРА...
Name: fullText, dtype: object
##################################################
1323
tomita
##################################################
finishrechec
[('НЕ ТОРМОЗИТ 2 АККУМУЛЯТОР ДОЛГОИГРАЮЩИЙ 3 4G ВЕЗДЕ ЛОВИТ 4 ОТЛИЧНЫЙ ЗВУК 5 ХОРОШАЯ КАМЕРА 6 ANDROID LOLLIPOP 5.1 ПОЗВОЛЯЕТ ПРОСМОТРЕТЬ И ЗАНЯТОСТИ ОЗУ И ЗАКРЫТЬ ЛИШНИЕ ПРИЛОЖЕНИЕ', 0.749177363962512), ('ИМЕТЬ БОЛЬШУЮ ЁМКОСТЬ БАТАРЕИ КОТОРОЙ ХВАТАЕТ НА 1,5 ДНЯ ЭТО ЕЩЁ АККУМУЛЯТОР НОРМАЛЬНО НЕ ЗАРЯЖАЛСЯ ПОЛНОСТЬЮ ВЕСЬ ЦИКЛ', 0.67133437687934), ('ВЕСЬ ПРОЧЕМУ В МАГАЗИНЕ КАК ОБЫЧНО НЕТ НИКАКИХ АКСЕССУАРОВ К ТАКИМ БЮДЖЕТНЫМ ТЕЛЕФОН', 0.5387976730097621), ('ТЕЛЕФОН ПАР РАЗ ЗАВИСАЛ НА ПУСТОМ МЕСТЕ ЗАВИСАЛ НА ГЛУХО НО ЭТО ПО ХОДУ ПРОБЛЕМА СЛАБОГО', 0.5207173823853711), ('ОДИН ОГРОМНЫЙ ПЛЮС ЭТО ЗАРЯДКУ ДЕРЖИТ ОЧЕНЬ ДОЛГО', 0.489820380081077), ('ПОЛЬЗОВАТЬСЯ ТРЕТИЙ ДЕНЬ И ПОКА НЕ НАШЛА НИ ОДНОГО МИНУС', 0.4670416062879946), ('БАТА

In [ ]:
ind = 1003
#print(textAndProdGrS[textColName].iloc[ind])
tomitaCol[ind]

In [ ]:
tfidf = fex.TfidfVectorizer()

In [ ]:
tfIdfMatrix = tfidf.fit_transform(textAndProdGrSnORM)
ftNames = tfidf.get_feature_names()

In [ ]:
densM = tfIdfMatrix.todense()
densM.shape

In [ ]:
sublist = [x[::, 0] for x in tomitaCol.values]

In [ ]:
slAll=list()
for sl in sublist:
    type(sl)
    slAll.append(list(sl))

In [ ]:
sublistInd = [i for i,n in enumerate(ftNames)  if n.upper() in slAll  ]

In [ ]:
sublistInd